In [ ]:
import requests
import json
import pandas as pd

class dadosAbertosSetorEletrico:
    def __init__(self, instituicao: str):
        
        self.api = '/api/3/action/'
        if str.lower(instituicao) == "ccee":
            self.host = 'https://dadosabertos.ccee.org.br'
        elif str.lower(instituicao) == "ons":
            self.host = 'https://dados.ons.org.br'
        elif str.lower(instituicao) == "aneel":
            self.host = 'https://dadosabertos.aneel.gov.br/'
        else:
            print("Instituição não encontrada!")

    def listar_produtos_disponiveis(self):
        r = requests.get(self.host+self.api+f"package_list")
        return r.json()

    def __buscar_resource_ids_por_produto(self, produto: str):
        r = requests.get(self.host+self.api+f"package_show?id={produto}")
        ids = [item['id'] for item in r.json()['result']['resources'] if 'id' in item]
        return ids


    def baixar_dados_produto_completo(self, produto: str):
        limite = 10000
        lista_dfs = []
        print("Preparando para baixar os arquivos...")
        
        for key in self.__buscar_resource_ids_por_produto(produto):
            offset = 0
            while True:
                r = requests.get(
                    self.host + self.api + f"datastore_search?resource_id={key}&limit={limite}&offset={offset}"
                )
                response = r.json()

                if not response.get("success", False) or "result" not in response or "records" not in response["result"]:
                    print(f"Recurso {key} não está disponível via API estruturada.")
                    print("Faça o download manual pelo portal de dados abertos.")
                    print(f"URL do recurso: {self.host}/dataset/{produto}")
                    break  # quebra o while, vai para o próximo resource_id

                registros = response["result"]["records"]
                if not registros:
                    break

                df = pd.DataFrame(registros)
                lista_dfs.append(df)

                offset += limite
                if offset >= response["result"].get("total", 0):
                    break
        
        return pd.concat(lista_dfs, ignore_index=True) if lista_dfs else None


carga = dadosAbertosSetorEletrico("CCEE")
# carga.listar_produtos_disponiveis()
carga.baixar_dados_produto_completo("ccen_custo_caft")

Preparando para baixar os arquivos...


,_id,MES_REFERENCIA,CUSTO_ADM_FINAN_TRIBUT_CCEN
0,1,202312,15334.79
1,2,202311,15165.22
2,3,202310,21457.74
3,4,202309,14137.99
4,5,202308,NaN
5,6,202307,14175.42
6,1,202412,74898.56
7,2,202411,11551.43
8,3,202410,10450.40
9,4,202409,10647.91
